# Importing libraries

In [ ]:
# models
from sklearn import linear_model,svm,tree,ensemble,naive_bayes,neighbors,discriminant_analysis
from sklearn import model_selection, preprocessing, metrics, feature_selection

# for unbalanced data
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import ADASYN,BorderlineSMOTE,KMeansSMOTE,RandomOverSampler,SMOTE,SMOTENC,SVMSMOTE


from scipy import stats as st
import pandas as pd
import numpy as np

import os
import pickle
import mifs
from copy import copy

from utils import get_model, get_selected_features
import warnings
warnings.filterwarnings('ignore')

# Read the dataset and preprocessing

In [ ]:
# define the training cohort and the testing cohort
train_data = pd.read_excel('Path to the training data')
test_data = pd.read_excel('Path to the internal testing data')
ext_data = pd.read_excel('Path to the external testing data')
test_data_list = [test_data,ext_data]

In [ ]:
# Analyze the distribution of data
print(train_data['label'].value_counts())
print(test_data['label'].value_counts())
print(ext_data['label'].value_counts())

# As the positive and negative samples in this study exhibit a relatively balanced distribution, 
# resampling of positive and negative samples is not utilized.

In [1]:
# Configure all feature names
feature_names = ['gender', 'age', 'jaundice', 'pancreatitis_history', 'CEA_level', 'CEA_elevating', 'CA199_level', 'CA199_elevating'] +\
                ['tumor_location', 'tumor_size', 'MPD_communication', 'diameter_MPD', 
                 'dilation_MPD', 'thick_cystic_wall', 'enhanced_cystic_wall', 'septations', 'thick_septations', 
                 'enhanced_septations', 'mural_nodule_solid_component', 'peripheral_calcification','central_calcification'] 

len(feature_names)

21

In [ ]:
# Get number of total features in feature_names list
featurn_num = len(feature_names) 

# # According to the TRIPOD guideline, the 10 EPV rule was used for the feature num choice 
select_max_num = min(sum(train_data['label']==0)//10, sum(train_data['label']==1)//10)

# Limit the maximum number of features to be selected to the number of total features or select_max_num, whichever is smaller.
select_max_num = min(featurn_num, select_max_num)

# Create a list of integers ranging from 1 to select_max_num as the number of features to consider
n_features = list(range(1, select_max_num+1))


In [ ]:
# preprocessing ... Delete irrelevant columns from the dataframe
train_X = train_data[feature_names].copy()
train_y = train_data['label'].copy()

test_Xs = []
test_ys = []
for each_test in test_data_list:
    test_Xs.append(each_test[feature_names].copy()) 
    test_ys.append(each_test['label'].copy())

print(train_X.shape)
for each_test in test_Xs:
    print(each_test.shape)

In [ ]:
# Create an instance of the StandardScaler from the preprocessing module
mms = preprocessing.StandardScaler()

# Select categorical and numerical features separately from feature_names list
cat_features = [f for f in feature_names if len(train_X[f].unique())<3]
num_features = [f for f in feature_names if f not in cat_features]

# Fit and transform the numerical features of train_X using StandardScaler, store as DataFrame and concatenate with categorical features
train_X_num = pd.DataFrame(mms.fit_transform(train_X[num_features]), columns=num_features)
train_X = pd.concat([train_X[cat_features].reset_index(drop=True), train_X_num], axis=1)

# Transform the numerical features of each test set using the same StandardScaler instance used for train_X
for i in range(len(test_Xs)):
    test_Xs_num = pd.DataFrame(mms.transform(test_Xs[i][num_features]), columns=num_features)
    test_Xs[i] = pd.concat([test_Xs[i][cat_features].reset_index(drop=True), test_Xs_num], axis=1)
    
# Print the shape of train_X and the shape of each test set after transformation
print(train_X.shape)
for each_test in test_Xs:
    print(each_test.shape)


# Model Config

In [ ]:
config = {
    'model_include': ['Logistic Regression','SVM','Decision Tree','Bernoulli NB','Gaussian NB','K Nearest Neighbors',
                   'Linear Discriminant Analysis'],
    'feature_selection_include':['JMIM','MRMR','JMI','F_Test','L1_based'],
    
    'save_dir':'Path to the output folder',
    
    'feature_selection':{
        'n_features':n_features,
    },
    b
    'model_params': {
        'Logistic Regression':{
            'penalty':['l1', 'l2', 'elasticnet', 'none'],
            'dual':[True,False],
            'C':[0.1, 0.5, 1, 5, 10],
            "fit_intercept": [True, False],
            "class_weight": ["balanced", None], 
            'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],

        },
        'SVM':{
            'C':[0.1, 0.5, 1],
            'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
            'degree':[2,3,4],
            'gamma':['scale','auto'],
            'class_weight':['balanced',None],

        },
        'Decision Tree':{
            'criterion':['gini', 'entropy'],
            'max_depth':[2,3,4,None],
            'class_weight':['balanced',None],
        },
        'Random Forest':{
            'n_estimators':[10, 50, 100],
            'criterion':['gini', 'entropy'],
            'max_depth':[2,3,4,None],
            'class_weight':['balanced','balanced_subsample',None],
            'bootstrap':[True,False]
        },
        'Ada Boost':{
            'n_estimators':[10, 50, 100],
            'learning_rate':[1e-4, 1e-3, 1e-2, 0.1],
            'algorithm':['SAMME','SAMME.R']
                             
        },
        'Gradient Boosting':{
            'loss':['deviance', 'exponential'],
            'learning_rate':[1e-4, 1e-3, 1e-2, 0.1],
            'n_estimators':[10, 50, 100],
            'criterion':['friedman_mse', 'mse', 'mae'],
            'max_depth':[2,3,4,None],
        },
        'XG Boost':{
             "booster": ["gbtree", "dart", "gblinear"], 
        },
        
        'Bernoulli NB':{
            'alpha':[1e-4, 1e-3, 1e-2, 0.1, 0.5, 1, 5, 10],
            "fit_prior": [True, False],
        },
        'Gaussian NB': {
        },
        'K Nearest Neighbors':{
            "n_neighbors": [2,3,4,5],
            "weights": ["uniform", "distance"],
            "p": [1, 2],
        },
        "Linear Discriminant Analysis": {
            "solver": ["svd", "lsqr"],
        },
        
    },
    
}

In [ ]:
# Model calculate metrics
def calculate(score, label, th):
    score = np.array(score)
    label = np.array(label)
    pred = np.zeros_like(label)
    pred[score>=th]=1
    pred[score<th]=0
    
    TP = len(pred[(pred>0.5) & (label>0.5)])
    FN = len(pred[(pred<0.5) & (label>0.5)])
    TN = len(pred[(pred<0.5) & (label<0.5)])
    FP = len(pred[(pred>0.5) & (label<0.5)])
    
    AUC = metrics.roc_auc_score(label, score)
    ACC = metrics.accuracy_score(label, pred)
    F1 = metrics.f1_score(label,pred)
    AP = metrics.average_precision_score(label, score)
    result = {'AUC':AUC, 'acc': ACC, 'F1': F1, 
              'sen':(TP)/(TP+FN+0.0001),'spe':(TN)/(TN+FP+0.0001),
             'AP':AP}
#     print('acc',(TP+TN),(TP+TN+FP+FN),'spe',(TN),(TN+FP),'sen',(TP),(TP+FN))
    return result

# Training and validating 

The model reached the performance (AUC score) in the cross-validation procedure was selected 

In [ ]:
# This may take long time...please remain patience 

# Initialize an empty list to store the results for each feature selection and model combination.
rst_dfs = []

# Loop over each number of features specified in the config file.
for feat_num in config['feature_selection']['n_features']:
    
    # Loop over each feature selector specified in the config file.
    for feat_selector in config['feature_selection_include']:
        try:
            # Get a list of selected features using the current feature selector, number of features, 
            # and training data. Subset the training data to include only the selected features.
            selected = get_selected_features(feat_selector, feature_names, feat_num, train_X, train_y)
            train_data_X = train_X[selected]
            
            # Print the number of features and feature selector being used.
            print('select...... %s ..... features  by %s' % (feat_num, feat_selector))
            print(selected)
            
            # Loop over each model specified in the config file.
            for model_name in config['model_include']:

                # Initialize an ADASYN sampler and obtain the model and its associated parameters using the 
                # current model name and config file. Fit the model to the training data with cross-validation 
                # using grid search to determine the best hyperparameters.
                sampler = ADASYN()
                model, params = get_model(model_name, config, False, sampler) # not include sampler
                rskf = model_selection.RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=0)
                grid = model_selection.GridSearchCV(model, params, scoring='roc_auc',cv = rskf, refit=True)
                grid.fit(train_data_X, train_y)
                
                # Obtain the best estimator from the grid search and perform cross-validation on the training data. 
                # Use the indices obtained from the cross-validation to obtain the predictions for each sample
                # in the training data.
                fitted_model = grid.best_estimator_
                cv_results  = model_selection.cross_validate(estimator=fitted_model, X = train_data_X, y = train_y,
                                                            cv = rskf, n_jobs=5, return_train_score=True, 
                                                             return_estimator=True, verbose=False)
                idxes_0 = []
                idxes_1 = []
                for idx_0, idx_1 in rskf.split(train_data_X,train_y):
                    idxes_0.append(idx_0)
                    idxes_1.append(idx_1)   
                rst_dict = {}
                for i in range(len(idxes_1)):
                    train_data_cv = train_data_X.iloc[idxes_1[i]]
                    model = cv_results['estimator'][i]
                    rst = model.predict_proba(train_data_cv)[:,1]
                    for idx, idy in enumerate(idxes_1[i]):
                        if idy not in rst_dict:
                            rst_dict[idy] = []
                        rst_dict[idy].append(rst[idx])

                # Compute the average validation score and obtain the predictions for each test set using the 
                # selected features and best estimator obtained from grid search. Save the results to disk 
                # in CSV format.
                val_score = np.mean(pd.DataFrame(rst_dict)[list(range(len(train_data_X)))])
                train_score = fitted_model.predict_proba(train_data_X)[:,1]
                test_scores = []
                for test_X in test_Xs:
                    test_X = test_X[selected]
                    test_score = fitted_model.predict_proba(test_X)[:,1]
                    test_scores.append(test_score)

                out_put_dir = os.path.join(config['save_dir'], model_name+'_%s_feat_%s' % (feat_num, feat_selector))
                os.makedirs(out_put_dir, exist_ok=True)

                # Save results for training, validation, and test sets as csv files
                train_rst = pd.DataFrame({'score':train_score, 'label':train_y})
                val_rst = pd.DataFrame({'score':val_score, 'label':train_y})
                for i, test_score in enumerate(test_scores):
                    test_rst = pd.DataFrame({'score':test_score, 'label':test_ys[i]})
                    test_rst.to_csv(os.path.join(out_put_dir, 'test_rst_%s.csv' % i), index=False)

                train_rst.to_csv(os.path.join(out_put_dir, 'train_rst.csv'), index=False)
                val_rst.to_csv(os.path.join(out_put_dir, 'val_rst.csv'), index=False)

                # Save the fitted model to a file
                pickle.dump(fitted_model, open(os.path.join(out_put_dir, 'model.pkl'),'wb'))

                # Calculate the performance metrics for the training, validation, and test sets
                fpr_micro, tpr_micro, th = metrics.roc_curve(train_y, val_score)
                max_th = -1
                max_yd = -1
                for i in range(lben(th)):
                    yd = tpr_micro[i]-fpr_micro[i]
                    if yd > max_yd:
                        max_yd = yd
                        max_th = th[i]

                rst_train = pd.DataFrame([calculate(train_score, train_y, max_th)])[['AUC','acc','F1','sen','spe','AP']]
                rst_val = pd.DataFrame([calculate(val_score, train_y, max_th)])[['AUC','acc','F1','sen','spe','AP']]

                rst_tests = []
                for i, test_score in enumerate(test_scores):
                    rst_test = pd.DataFrame([calculate(test_score, test_ys[i], max_th)])[['AUC','acc','F1','sen','spe','AP']]
                    rst_tests.append(rst_test)

                # Combine the performance metrics for the training, validation, and test sets into a single dataframe
                all_rst = [rst_train, rst_val]
                all_rst.extend(rst_tests)
                rst = pd.concat(all_rst, axis=1)  
                rst['model'] = model_name
                rst['feat_num'] = feat_num
                rst['feature_selector'] = feat_selector
                rst_dfs.append(rst)
            # Save the selected features to a file
            pd.DataFrame({'feat':selected}).to_csv(os.path.join(config['save_dir'], 'feat_%s_%s.csv' % (feat_num, feat_selector)))
        except:
            continue
            
# Combine the results for all feature selection settings and models into a single dataframe
rst_df = pd.concat(rst_dfs)

# Rename the columns of the dataframe
columns = ['AUC_train', 'acc_train', 'F1_train', 'sen_train', 'spe_train', 'AP_train',
           'AUC_val', 'acc_val', 'F1_val', 'sen_val', 'spe_val', 'AP_val']+\
            ['%s_test_%s' % (f,v) for v in range(len(test_Xs)) for f in ['AUC','acc','F1','sen','spe','AP'] ]+\
    ['model','feat_num','feature_selector']
rst_df = pd.DataFrame(rst_df.values, columns=columns)
# Save the dataframe to a file
rst_df.to_csv(os.path.join(config['save_dir'], 'model_pf.csv'))

# Get the best performing model for each model type based on the validation set AUC
best_df = rst_df.groupby('model').apply(lambda x:x.sort_values(by='AUC_val', ascending=False).iloc[0]).reset_index(drop=True)
best_df.sort_values(by='AUC_val', ascending=False, inplace=True)
best_df.to_csv(os.path.join(config['save_dir'], '_best_models_%s.csv' % config['save_dir'].split('/')[-1]))  